In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.ticker as mtick
import arcpy

# Import the required ArcGIS API for Python modules
import arcgis
gis = arcgis.GIS()

from IPython.display import display, Markdown

In [2]:
#get root folder
dir_working = os.getcwd()
print(dir_working)

#define folders
dir_data     = os.path.join(dir_working, "data"        )
dir_process  = os.path.join(dir_working, "intermediate")
dir_results  = os.path.join(dir_working, "results"     )
dir_inputs   = os.path.join(dir_working, "input"       )

ClassParameters    = os.path.join(dir_inputs, r"class_parameters.csv"     )
CountiesTable      = os.path.join(dir_inputs ,r"counties.csv"             )

ProcessGDB = "process.gdb"

#name of new data features
AnalysisAreas      = os.path.join(dir_process, "analysisareas.shp"        )
AnalysisAreas_IDs  = os.path.join(dir_process, "analysisareas_ids.shp"    )
ParcelsAA          = os.path.join(dir_process, "parcelsaa.shp"            )

AnalysisAreasTable = os.path.join(dir_results, "areas.csv"                )



def deleteIfExists(obj):
    if arcpy.Exists(obj): arcpy.Delete_management(obj)


E:\GitHub\Residential-Capacity


In [3]:
AnalysisAreas

'E:\\GitHub\\Residential-Capacity\\intermediate\\analysisareas.shp'

In [4]:
sdf_AA = pd.DataFrame.spatial.from_featureclass(AnalysisAreas)
#sdf_AA

sdf_AAid = pd.DataFrame.spatial.from_featureclass(AnalysisAreas_IDs)
sdf_AAid

,FID,Shape_Leng,Shape_Area,AreaID,ClassID,SHAPE
0,0,599.400903,1.220382e+04,4,a4,"{""rings"": [[[420271.3705000002, 4480517.338500..."
1,1,1554.841774,8.561669e+04,4,a4,"{""rings"": [[[420636.9475999996, 4481132.772299..."
2,2,254.967270,3.811046e+03,4,a4,"{""rings"": [[[420671.3481999999, 4481261.932600..."
3,3,2666.392431,2.609549e+05,4,a4,"{""rings"": [[[420869.50710000005, 4482955.5603]..."
4,4,11708.780939,1.126298e+06,4,a4,"{""rings"": [[[421226.04870000016, 4479294.0206]..."
...,...,...,...,...,...,...
11371,11371,8681.643024,8.610622e+05,10,g4,"{""rings"": [[[414009.90950000007, 4551759.66699..."
11372,11372,2499.500722,2.280914e+05,10,g4,"{""rings"": [[[414770.7604, 4550604.501800001], ..."
11373,11373,5944.802252,9.146734e+05,45,g3,"{""rings"": [[[413607.37820000015, 4593065.92], ..."
11374,11374,694.106922,2.660076e+04,45,g2,"{""rings"": [[[414181.2357999999, 4591852.214600..."


In [5]:
##create map centered on Salt Lake
#map_areas = gis.map('Salt Lake')
#
##expression to classify enrollment
#arcade_expression_aa = ("return 'classAA';")
#
##symbology for enrollment classes
#uv_aa = [{"value":"classAA", "label":"Analysis Area", "symbol":{"type":"esriSFS","color":[128,128,128,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}}]
#
##classify land use
#arcade_expression_aa = ("var v = $feature.Analysis_1;"
#                        "if      (v=='NONTOD|Metropolitan Center') { return 'mc'    ; }"
#                        "else if (v=='CRT|Metropolitan Center'   ) { return 'mc_tod'; }"
#                        "else if (v=='LRT|Metropolitan Center'   ) { return 'mc_tod'; }"
#                        "else if (v=='NONTOD|Urban Center'       ) { return 'uc'    ; }"
#                        "else if (v=='CRT|Urban Center'          ) { return 'uc_tod'; }"
#                        "else if (v=='LRT|Urban Center'          ) { return 'uc_tod'; }"
#                        "else if (v=='NONTOD|City Center'        ) { return 'cc'    ; }"
#                        "else if (v=='CRT|City Center'           ) { return 'cc_tod'; }"
#                        "else if (v=='LRT|City Center'           ) { return 'cc_tod'; }"
#                        "else if (v=='CRT|NA'                    ) { return 'nc_tod'; }"
#                        "else if (v=='LRT|NA'                    ) { return 'nc_tod'; }"
#                        "else                                      { return 'other' ; }")
#
##symbology for enrollment classes
#uv_aa = [  
#          {"value":"mc"    , "label":"Metropolitan Center"         , "symbol":{"type":"esriSFS","color":[128,  0,128,100], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
#          {"value":"mc_tod", "label":"Metropolitan Center with TOD", "symbol":{"type":"esriSFS","color":[128,  0,128,200], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
#          {"value":"uc"    , "label":"Urban Center"                , "symbol":{"type":"esriSFS","color":[  0,  0,128,100], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
#          {"value":"uc_tod", "label":"Urban Center with TOD"       , "symbol":{"type":"esriSFS","color":[  0,  0,128,200], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
#          {"value":"cc"    , "label":"City Center"                 , "symbol":{"type":"esriSFS","color":[255,  0,  0,100], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
#          {"value":"cc_tod", "label":"City Center with TOD"        , "symbol":{"type":"esriSFS","color":[255,  0,  0,200], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
#          {"value":"nc_tod", "label":"TOD Only"                    , "symbol":{"type":"esriSFS","color":[  0,255,  0,200], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
#          {"value":"other" , "label":"Other"                       , "symbol":{"type":"esriSFS","color":[128,128,128,200], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}}
#        ]
#
##define sdf layer
#sdf_AA.spatial.plot(map_widget = map_areas,
#                                 renderer_type='u-a', #'u-a' stands for uniqe value with arcade expression
#                                 unique_values=uv_aa,
#                                 arcade_expression=arcade_expression_aa,
#                                 default_symbol="" #don't include an 'other' category
#                                 )
#
##define map characteristics
#map_areas.layout.height='500px'
#map_areas.legend=True
#
##map title
#display(Markdown('<h2><center>Analysis Areas</center></h2>'))
#
##display map
#map_areas

# Parameters

In [6]:
df_ClassParam = pd.read_csv(ClassParameters)
display(df_ClassParam)

,ClassID,ClassDescription,SFSplitRes,SFSplitCom,CapacityRes_DUA,CapacityCom_FAR,SFperHH,SFperEmp,PercentOpenSpace,RedevValuePerAcreRes,RedevValuePerAcreCom,RedevAndOr,RedevBldgAgeRes_Low,RedevBldgAgeRes_High,RedevBldgAgeCom_Low,RedevBldgAgeCom_High,RedevProb,SFRedevFullAdd,ClassOrder
0,f4,Multifamily Metro,0.40,0.60,120.0,15.0,1500,225,0.10,5000000,4000000,AND,60,120,20,120,1,No,29
1,f3,Multifamily City/TOD,0.50,0.50,60.0,4.0,1500,250,0.10,3000000,2400000,AND,60,120,20,120,1,No,28
2,f2,Multifamily Suburban,0.70,0.30,30.0,1.0,1500,300,0.10,1500000,1200000,AND,60,120,20,120,1,No,27
3,f1,Multifamily Single Story,0.85,0.15,10.0,0.5,2000,300,0.10,1000000,800000,AND,60,300,20,300,1,No,26
4,g4,Mixed-Use Metro,0.40,0.60,120.0,15.0,1500,225,0.10,5000000,4000000,AND,60,120,20,120,1,No,33
5,g3,Mixed-Use City/TOD,0.50,0.50,60.0,4.0,1500,250,0.10,3000000,2400000,AND,60,120,20,120,1,No,32
6,g2,Mixed-Use Suburban,0.70,0.30,30.0,1.0,1500,300,0.10,1500000,1200000,AND,60,120,20,120,1,No,31
7,g1,Mixed-Use Single Story,0.85,0.15,10.0,0.5,2000,300,0.10,1000000,800000,AND,60,300,20,300,1,No,30
8,h4,Mixed Residential Metro,0.40,0.60,120.0,15.0,1500,225,0.10,5000000,4000000,AND,60,120,20,120,1,No,37
9,h3,Mixed Residential City/TOD,0.50,0.50,60.0,4.0,1500,250,0.10,3000000,2400000,AND,60,120,20,120,1,No,36


In [7]:
df_ClassParam.dtypes

ClassID                  object
ClassDescription         object
SFSplitRes              float64
SFSplitCom              float64
CapacityRes_DUA         float64
CapacityCom_FAR         float64
SFperHH                   int64
SFperEmp                  int64
PercentOpenSpace        float64
RedevValuePerAcreRes      int64
RedevValuePerAcreCom      int64
RedevAndOr               object
RedevBldgAgeRes_Low       int64
RedevBldgAgeRes_High      int64
RedevBldgAgeCom_Low       int64
RedevBldgAgeCom_High      int64
RedevProb                 int64
SFRedevFullAdd           object
ClassOrder                int64
dtype: object

In [8]:
df_AADevCat = pd.read_csv(os.path.join(dir_results, r'AnalysisAreas_byDevCategory.csv'))
df_AADevCat

,AreaID,ClassID,county_id,DevCategory,parcel_id,job_spaces,resunits,Acres,sf_res,sf_com,acres_res,acres_com
0,0,a1,0.0,Developable,18.0,0.000000,0.000000,15.567788,0.000000,0.000000,0.000000,0.000000
1,0,a1,0.0,Undevelopable,8.0,0.000000,0.000000,14.733847,0.000000,0.000000,0.000000,0.000000
2,0,a1,1.0,Undevelopable,1.0,0.000000,0.000000,0.108287,0.000000,0.000000,0.000000,0.000000
3,0,a1,2.0,Developable,102.0,0.000000,0.000000,234.281314,0.000000,0.000000,0.000000,0.000000
4,0,a1,2.0,Redevelopable,10.0,0.043236,8.002301,84.596637,10868.498573,42.155347,83.663641,0.932996
...,...,...,...,...,...,...,...,...,...,...,...,...
2549,79,g2,1.0,Remain,2.0,0.000005,0.000000,0.004237,0.000000,0.003396,0.000000,0.004237
2550,79,g2,1.0,Undevelopable,1.0,0.000000,0.000000,0.197377,0.000000,0.000000,0.000000,0.000000
2551,79,g3,1.0,Developable,28.0,0.000000,0.000000,80.238874,0.000000,0.000000,0.000000,0.000000
2552,79,g3,1.0,Redevelopable,1.0,0.000000,0.998350,0.320457,1186.039267,0.000000,0.320457,0.000000


# Calculate New Jobs/HH

In [9]:
#join Analysis Areas with class parameters
df_AADevCatwParam = pd.DataFrame.merge(df_AADevCat,df_ClassParam,on='ClassID',how='left')
df_AADevCatwParam

,AreaID,ClassID,county_id,DevCategory,parcel_id,job_spaces,resunits,Acres,sf_res,sf_com,...,RedevValuePerAcreRes,RedevValuePerAcreCom,RedevAndOr,RedevBldgAgeRes_Low,RedevBldgAgeRes_High,RedevBldgAgeCom_Low,RedevBldgAgeCom_High,RedevProb,SFRedevFullAdd,ClassOrder
0,0,a1,0.0,Developable,18.0,0.000000,0.000000,15.567788,0.000000,0.000000,...,40000,1000000,AND,50,300,20,300,1,Yes,1
1,0,a1,0.0,Undevelopable,8.0,0.000000,0.000000,14.733847,0.000000,0.000000,...,40000,1000000,AND,50,300,20,300,1,Yes,1
2,0,a1,1.0,Undevelopable,1.0,0.000000,0.000000,0.108287,0.000000,0.000000,...,40000,1000000,AND,50,300,20,300,1,Yes,1
3,0,a1,2.0,Developable,102.0,0.000000,0.000000,234.281314,0.000000,0.000000,...,40000,1000000,AND,50,300,20,300,1,Yes,1
4,0,a1,2.0,Redevelopable,10.0,0.043236,8.002301,84.596637,10868.498573,42.155347,...,40000,1000000,AND,50,300,20,300,1,Yes,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2549,79,g2,1.0,Remain,2.0,0.000005,0.000000,0.004237,0.000000,0.003396,...,1500000,1200000,AND,60,120,20,120,1,No,31
2550,79,g2,1.0,Undevelopable,1.0,0.000000,0.000000,0.197377,0.000000,0.000000,...,1500000,1200000,AND,60,120,20,120,1,No,31
2551,79,g3,1.0,Developable,28.0,0.000000,0.000000,80.238874,0.000000,0.000000,...,3000000,2400000,AND,60,120,20,120,1,No,32
2552,79,g3,1.0,Redevelopable,1.0,0.000000,0.998350,0.320457,1186.039267,0.000000,...,3000000,2400000,AND,60,120,20,120,1,No,32


In [10]:
#remove percent open space from available acreage
df_AADevCatwParam['AcresOpenSpaceRemoved'] = df_AADevCatwParam['Acres'] * (1-df_AADevCatwParam['PercentOpenSpace'])
pd.set_option('display.max_columns', None)
display(df_AADevCatwParam)
display(df_AADevCatwParam.columns)

,AreaID,ClassID,county_id,DevCategory,parcel_id,job_spaces,resunits,Acres,sf_res,sf_com,acres_res,acres_com,ClassDescription,SFSplitRes,SFSplitCom,CapacityRes_DUA,CapacityCom_FAR,SFperHH,SFperEmp,PercentOpenSpace,RedevValuePerAcreRes,RedevValuePerAcreCom,RedevAndOr,RedevBldgAgeRes_Low,RedevBldgAgeRes_High,RedevBldgAgeCom_Low,RedevBldgAgeCom_High,RedevProb,SFRedevFullAdd,ClassOrder,AcresOpenSpaceRemoved
0,0,a1,0.0,Developable,18.0,0.000000,0.000000,15.567788,0.000000,0.000000,0.000000,0.000000,Single Family A1,1.0,0.0,1.5,0.5,4000,350,0.15,40000,1000000,AND,50,300,20,300,1,Yes,1,13.232620
1,0,a1,0.0,Undevelopable,8.0,0.000000,0.000000,14.733847,0.000000,0.000000,0.000000,0.000000,Single Family A1,1.0,0.0,1.5,0.5,4000,350,0.15,40000,1000000,AND,50,300,20,300,1,Yes,1,12.523770
2,0,a1,1.0,Undevelopable,1.0,0.000000,0.000000,0.108287,0.000000,0.000000,0.000000,0.000000,Single Family A1,1.0,0.0,1.5,0.5,4000,350,0.15,40000,1000000,AND,50,300,20,300,1,Yes,1,0.092044
3,0,a1,2.0,Developable,102.0,0.000000,0.000000,234.281314,0.000000,0.000000,0.000000,0.000000,Single Family A1,1.0,0.0,1.5,0.5,4000,350,0.15,40000,1000000,AND,50,300,20,300,1,Yes,1,199.139117
4,0,a1,2.0,Redevelopable,10.0,0.043236,8.002301,84.596637,10868.498573,42.155347,83.663641,0.932996,Single Family A1,1.0,0.0,1.5,0.5,4000,350,0.15,40000,1000000,AND,50,300,20,300,1,Yes,1,71.907142
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2549,79,g2,1.0,Remain,2.0,0.000005,0.000000,0.004237,0.000000,0.003396,0.000000,0.004237,Mixed-Use Suburban,0.7,0.3,30.0,1.0,1500,300,0.10,1500000,1200000,AND,60,120,20,120,1,No,31,0.003813
2550,79,g2,1.0,Undevelopable,1.0,0.000000,0.000000,0.197377,0.000000,0.000000,0.000000,0.000000,Mixed-Use Suburban,0.7,0.3,30.0,1.0,1500,300,0.10,1500000,1200000,AND,60,120,20,120,1,No,31,0.177639
2551,79,g3,1.0,Developable,28.0,0.000000,0.000000,80.238874,0.000000,0.000000,0.000000,0.000000,Mixed-Use City/TOD,0.5,0.5,60.0,4.0,1500,250,0.10,3000000,2400000,AND,60,120,20,120,1,No,32,72.214986
2552,79,g3,1.0,Redevelopable,1.0,0.000000,0.998350,0.320457,1186.039267,0.000000,0.320457,0.000000,Mixed-Use City/TOD,0.5,0.5,60.0,4.0,1500,250,0.10,3000000,2400000,AND,60,120,20,120,1,No,32,0.288411


Index(['AreaID', 'ClassID', 'county_id', 'DevCategory', 'parcel_id',
       'job_spaces', 'resunits', 'Acres', 'sf_res', 'sf_com', 'acres_res',
       'acres_com', 'ClassDescription', 'SFSplitRes', 'SFSplitCom',
       'CapacityRes_DUA', 'CapacityCom_FAR', 'SFperHH', 'SFperEmp',
       'PercentOpenSpace', 'RedevValuePerAcreRes', 'RedevValuePerAcreCom',
       'RedevAndOr', 'RedevBldgAgeRes_Low', 'RedevBldgAgeRes_High',
       'RedevBldgAgeCom_Low', 'RedevBldgAgeCom_High', 'RedevProb',
       'SFRedevFullAdd', 'ClassOrder', 'AcresOpenSpaceRemoved'],
      dtype='object')

In [11]:
#calculate Com and Res Acreage

#initialize - acres_remain and acres_developed don't have res/com breakdown
df_AADevCatwParam['acres_undevelopable'] = 0.0
df_AADevCatwParam['acres_remain'       ] = 0.0
df_AADevCatwParam['acres_redevelopable'] = 0.0
df_AADevCatwParam['acres_developable'  ] = 0.0

#set acres
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Undevelopable'), 'acres_undevelopable'] = df_AADevCatwParam['Acres']
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Remain'       ), 'acres_remain'       ] = df_AADevCatwParam['Acres']
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Redevelopable'), 'acres_redevelopable'] = df_AADevCatwParam['Acres']
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Developable')  , 'acres_developable'  ] = df_AADevCatwParam['Acres']


#calculate Com and Res SF


#initialize
df_AADevCatwParam['SFcom_undevelopable'   ] = 0.0
df_AADevCatwParam['SFres_undevelopable'   ] = 0.0

df_AADevCatwParam['SFcom_remain'          ] = 0.0
df_AADevCatwParam['SFres_remain'          ] = 0.0

df_AADevCatwParam['SFcom_redeveloped_orig'] = 0.0
df_AADevCatwParam['SFres_redeveloped_orig'] = 0.0

df_AADevCatwParam['SFcom_redeveloped_new' ] = 0.0
df_AADevCatwParam['SFres_redeveloped_new' ] = 0.0

df_AADevCatwParam['SFcom_developed'       ] = 0.0
df_AADevCatwParam['SFres_developed'       ] = 0.0


#set undevelopable sf
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Undevelopable'), 'SFcom_undevelopable'     ] = df_AADevCatwParam['sf_com']
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Undevelopable'), 'SFres_undevelopable'     ] = df_AADevCatwParam['sf_res']

#set remaining sf
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Remain'       ), 'SFcom_remain'            ] = df_AADevCatwParam['sf_com']
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Remain'       ), 'SFres_remain'            ] = df_AADevCatwParam['sf_res']


# SFCom = AcresNoOpenSpace / ((SPLITres/SPLITcom)/(HHperAcre*SFperHH) + (1/(FAR*43560)))
# SFRes = SFCom * SPLITres/SPLITcom
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Redevelopable'), 'SFcom_redeveloped_orig'  ] = df_AADevCatwParam['sf_com']
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Redevelopable'), 'SFres_redeveloped_orig'  ] = df_AADevCatwParam['sf_res']


# temporarily set a very low number for zero to allow calcs to run

df_AADevCatwParam.loc[(df_AADevCatwParam['SFSplitCom'] == 0), 'SFSplitCom'] = 0.00000000001
df_AADevCatwParam.loc[(df_AADevCatwParam['SFSplitRes'] == 0), 'SFSplitRes'] = 0.00000000001

#Split Commercial and Split Residential > 0
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Redevelopable') & (df_AADevCatwParam['SFSplitCom'] >  0) & (df_AADevCatwParam['SFSplitRes'] >  0), 'SFcom_redeveloped_new'   ] = (df_AADevCatwParam['AcresOpenSpaceRemoved'] / ((df_AADevCatwParam['SFSplitRes']/df_AADevCatwParam['SFSplitCom'])/(df_AADevCatwParam['CapacityRes_DUA']*df_AADevCatwParam['SFperHH']) + (1/(df_AADevCatwParam['CapacityCom_FAR']*43560)))).astype('int64')
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Redevelopable') & (df_AADevCatwParam['SFSplitCom'] >  0) & (df_AADevCatwParam['SFSplitRes'] >  0), 'SFres_redeveloped_new'   ] = (df_AADevCatwParam['SFcom_redeveloped_new'] * df_AADevCatwParam['SFSplitRes'] / df_AADevCatwParam['SFSplitCom']).astype('int64')

df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Developable'  ) & (df_AADevCatwParam['SFSplitCom'] >  0) & (df_AADevCatwParam['SFSplitRes'] >  0), 'SFcom_developed'         ] = (df_AADevCatwParam['AcresOpenSpaceRemoved'] / ((df_AADevCatwParam['SFSplitRes']/df_AADevCatwParam['SFSplitCom'])/(df_AADevCatwParam['CapacityRes_DUA']*df_AADevCatwParam['SFperHH']) + (1/(df_AADevCatwParam['CapacityCom_FAR']*43560)))).astype('int64')
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Developable'  ) & (df_AADevCatwParam['SFSplitCom'] >  0) & (df_AADevCatwParam['SFSplitRes'] >  0), 'SFres_developed'         ] = (df_AADevCatwParam['SFcom_developed']       * df_AADevCatwParam['SFSplitRes'] / df_AADevCatwParam['SFSplitCom']).astype('int64')

df_AADevCatwParam.loc[(df_AADevCatwParam['SFSplitCom'] == 0.00000000001), 'SFSplitCom'] = 0
df_AADevCatwParam.loc[(df_AADevCatwParam['SFSplitRes'] == 0.00000000001), 'SFSplitRes'] = 0

#Split Commercial = 0
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Redevelopable') & (df_AADevCatwParam['SFSplitCom'] == 0), 'SFcom_redeveloped_new'   ] = 0
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Redevelopable') & (df_AADevCatwParam['SFSplitCom'] == 0), 'SFres_redeveloped_new'   ] = (df_AADevCatwParam['AcresOpenSpaceRemoved'] * df_AADevCatwParam['SFperHH'] * df_AADevCatwParam['CapacityRes_DUA']).astype('int64')

df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Developable'  ) & (df_AADevCatwParam['SFSplitCom'] == 0), 'SFcom_developed'         ] = 0
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Developable'  ) & (df_AADevCatwParam['SFSplitCom'] == 0), 'SFres_developed'         ] = (df_AADevCatwParam['AcresOpenSpaceRemoved'] * df_AADevCatwParam['SFperHH'] * df_AADevCatwParam['CapacityRes_DUA']).astype('int64')

#Split Residential = 0
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Redevelopable') & (df_AADevCatwParam['SFSplitRes'] == 0), 'SFcom_redeveloped_new'   ] = (df_AADevCatwParam['AcresOpenSpaceRemoved'] * df_AADevCatwParam['CapacityCom_FAR'] * 43560).astype('int64')
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Redevelopable') & (df_AADevCatwParam['SFSplitRes'] == 0), 'SFres_redeveloped_new'   ] = 0

df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Developable'  ) & (df_AADevCatwParam['SFSplitRes'] == 0), 'SFcom_developed'         ] = (df_AADevCatwParam['AcresOpenSpaceRemoved'] * df_AADevCatwParam['CapacityCom_FAR'] * 43560).astype('int64')
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Developable'  ) & (df_AADevCatwParam['SFSplitRes'] == 0), 'SFres_developed'         ] = 0

df_AADevCatwParam

,AreaID,ClassID,county_id,DevCategory,parcel_id,job_spaces,resunits,Acres,sf_res,sf_com,acres_res,acres_com,ClassDescription,SFSplitRes,SFSplitCom,CapacityRes_DUA,CapacityCom_FAR,SFperHH,SFperEmp,PercentOpenSpace,RedevValuePerAcreRes,RedevValuePerAcreCom,RedevAndOr,RedevBldgAgeRes_Low,RedevBldgAgeRes_High,RedevBldgAgeCom_Low,RedevBldgAgeCom_High,RedevProb,SFRedevFullAdd,ClassOrder,AcresOpenSpaceRemoved,acres_undevelopable,acres_remain,acres_redevelopable,acres_developable,SFcom_undevelopable,SFres_undevelopable,SFcom_remain,SFres_remain,SFcom_redeveloped_orig,SFres_redeveloped_orig,SFcom_redeveloped_new,SFres_redeveloped_new,SFcom_developed,SFres_developed
0,0,a1,0.0,Developable,18.0,0.000000,0.000000,15.567788,0.000000,0.000000,0.000000,0.000000,Single Family A1,1.0,0.0,1.5,0.5,4000,350,0.15,40000,1000000,AND,50,300,20,300,1,Yes,1,13.232620,0.000000,0.000000,0.000000,15.567788,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,79395.0
1,0,a1,0.0,Undevelopable,8.0,0.000000,0.000000,14.733847,0.000000,0.000000,0.000000,0.000000,Single Family A1,1.0,0.0,1.5,0.5,4000,350,0.15,40000,1000000,AND,50,300,20,300,1,Yes,1,12.523770,14.733847,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
2,0,a1,1.0,Undevelopable,1.0,0.000000,0.000000,0.108287,0.000000,0.000000,0.000000,0.000000,Single Family A1,1.0,0.0,1.5,0.5,4000,350,0.15,40000,1000000,AND,50,300,20,300,1,Yes,1,0.092044,0.108287,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
3,0,a1,2.0,Developable,102.0,0.000000,0.000000,234.281314,0.000000,0.000000,0.000000,0.000000,Single Family A1,1.0,0.0,1.5,0.5,4000,350,0.15,40000,1000000,AND,50,300,20,300,1,Yes,1,199.139117,0.000000,0.000000,0.000000,234.281314,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,1194834.0
4,0,a1,2.0,Redevelopable,10.0,0.043236,8.002301,84.596637,10868.498573,42.155347,83.663641,0.932996,Single Family A1,1.0,0.0,1.5,0.5,4000,350,0.15,40000,1000000,AND,50,300,20,300,1,Yes,1,71.907142,0.000000,0.000000,84.596637,0.000000,0.0,0.0,0.000000,0.000000,42.155347,10868.498573,0.0,431442.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2549,79,g2,1.0,Remain,2.0,0.000005,0.000000,0.004237,0.000000,0.003396,0.000000,0.004237,Mixed-Use Suburban,0.7,0.3,30.0,1.0,1500,300,0.10,1500000,1200000,AND,60,120,20,120,1,No,31,0.003813,0.000000,0.004237,0.000000,0.000000,0.0,0.0,0.003396,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
2550,79,g2,1.0,Undevelopable,1.0,0.000000,0.000000,0.197377,0.000000,0.000000,0.000000,0.000000,Mixed-Use Suburban,0.7,0.3,30.0,1.0,1500,300,0.10,1500000,1200000,AND,60,120,20,120,1,No,31,0.177639,0.197377,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
2551,79,g3,1.0,Developable,28.0,0.000000,0.000000,80.238874,0.000000,0.000000,0.000000,0.000000,Mixed-Use City/TOD,0.5,0.5,60.0,4.0,1500,250,0.10,3000000,2400000,AND,60,120,20,120,1,No,32,72.214986,0.000000,0.000000,0.000000,80.238874,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,4285674.0,4285674.0
2552,79,g3,1.0,Redevelopable,1.0,0.000000,0.998350,0.320457,1186.039267,0.000000,0.320457,0.000000,Mixed-Use City/TOD,0.5,0.5,60.0,4.0,1500,250,0.10,3000000,2400000,AND,60,120,20,120,1,No,32,0.288411,0.000000,0.000000,0.320457,0.000000,0.0,0.0,0.000000,0.000000,0.000000,1186.039267,17116.0,17116.0,0.0,0.0


In [12]:
df_AADevCatwParam[df_AADevCatwParam['AreaID']==52]

,AreaID,ClassID,county_id,DevCategory,parcel_id,job_spaces,resunits,Acres,sf_res,sf_com,acres_res,acres_com,ClassDescription,SFSplitRes,SFSplitCom,CapacityRes_DUA,CapacityCom_FAR,SFperHH,SFperEmp,PercentOpenSpace,RedevValuePerAcreRes,RedevValuePerAcreCom,RedevAndOr,RedevBldgAgeRes_Low,RedevBldgAgeRes_High,RedevBldgAgeCom_Low,RedevBldgAgeCom_High,RedevProb,SFRedevFullAdd,ClassOrder,AcresOpenSpaceRemoved,acres_undevelopable,acres_remain,acres_redevelopable,acres_developable,SFcom_undevelopable,SFres_undevelopable,SFcom_remain,SFres_remain,SFcom_redeveloped_orig,SFres_redeveloped_orig,SFcom_redeveloped_new,SFres_redeveloped_new,SFcom_developed,SFres_developed
1681,52,b2,3.0,Developable,9.0,0.000000e+00,0.000000,0.345956,0.000000e+00,0.000000e+00,0.000000,0.000000,Single Family B2,1.00,0.00,2.5,0.5,3500,350,0.15,125000,1000000,AND,50,300,20,300,1,Yes,7,0.294063,0.000000,0.000000,0.000000,0.345956,0.000000,0.000000,0.000000e+00,0.000000e+00,0.0,0.000000,0.0,0.0,0.0,2573.0
1682,52,b2,3.0,Redevelopable,6.0,0.000000e+00,5.394264,6.490133,6.527636e+03,0.000000e+00,6.490133,0.000000,Single Family B2,1.00,0.00,2.5,0.5,3500,350,0.15,125000,1000000,AND,50,300,20,300,1,Yes,7,5.516613,0.000000,0.000000,6.490133,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.0,6527.636278,0.0,48270.0,0.0,0.0
1683,52,b2,3.0,Remain,59.0,1.416127e-12,63.208106,49.133030,8.867787e+04,8.964082e-10,49.133021,0.000009,Single Family B2,1.00,0.00,2.5,0.5,3500,350,0.15,125000,1000000,AND,50,300,20,300,1,Yes,7,41.763075,0.000000,49.133030,0.000000,0.000000,0.000000,0.000000,8.964082e-10,8.867787e+04,0.0,0.000000,0.0,0.0,0.0,0.0
1684,52,b2,3.0,Undevelopable,3.0,0.000000e+00,0.000000,0.112441,0.000000e+00,0.000000e+00,0.000000,0.000000,Single Family B2,1.00,0.00,2.5,0.5,3500,350,0.15,125000,1000000,AND,50,300,20,300,1,Yes,7,0.095575,0.112441,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.0,0.000000,0.0,0.0,0.0,0.0
1685,52,b3,3.0,Developable,28.0,0.000000e+00,0.000000,24.563466,0.000000e+00,0.000000e+00,0.000000,0.000000,Single Family B3,1.00,0.00,2.5,0.5,3500,350,0.15,150000,1000000,AND,50,300,20,300,1,Yes,8,20.878946,0.000000,0.000000,0.000000,24.563466,0.000000,0.000000,0.000000e+00,0.000000e+00,0.0,0.000000,0.0,0.0,0.0,182690.0
1686,52,b3,3.0,Redevelopable,16.0,0.000000e+00,11.929072,16.816116,1.786910e+04,0.000000e+00,16.816116,0.000000,Single Family B3,1.00,0.00,2.5,0.5,3500,350,0.15,150000,1000000,AND,50,300,20,300,1,Yes,8,14.293699,0.000000,0.000000,16.816116,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.0,17869.100066,0.0,125069.0,0.0,0.0
1687,52,b3,3.0,Remain,483.0,2.698530e+02,409.644171,209.205270,6.947410e+05,1.814376e+05,196.368798,12.836472,Single Family B3,1.00,0.00,2.5,0.5,3500,350,0.15,150000,1000000,AND,50,300,20,300,1,Yes,8,177.824480,0.000000,209.205270,0.000000,0.000000,0.000000,0.000000,1.814376e+05,6.947410e+05,0.0,0.000000,0.0,0.0,0.0,0.0
1688,52,b3,3.0,Undevelopable,59.0,0.000000e+00,10.570028,14.972563,1.329240e+04,0.000000e+00,0.000000,0.000000,Single Family B3,1.00,0.00,2.5,0.5,3500,350,0.15,150000,1000000,AND,50,300,20,300,1,Yes,8,12.726679,14.972563,0.000000,0.000000,0.000000,0.000000,13292.396409,0.000000e+00,0.000000e+00,0.0,0.000000,0.0,0.0,0.0,0.0
1689,52,b4,3.0,Developable,1.0,0.000000e+00,0.000000,0.015011,0.000000e+00,0.000000e+00,0.000000,0.000000,Single Family B4,1.00,0.00,2.5,0.5,3500,350,0.15,250000,1000000,AND,50,300,20,300,1,Yes,9,0.012760,0.000000,0.000000,0.000000,0.015011,0.000000,0.000000,0.000000e+00,0.000000e+00,0.0,0.000000,0.0,0.0,0.0,111.0
1690,52,b4,3.0,Remain,14.0,0.000000e+00,6.983104,6.585786,1.351128e+04,0.000000e+00,6.585786,0.000000,Single Family B4,1.00,0.00,2.5,0.5,3500,350,0.15,250000,1000000,AND,50,300,20,300,1,Yes,9,5.597918,0.000000,6.585786,0.000000,0.000000,0.000000,0.000000,0.000000e+00,1.351128e+04,0.0,0.000000,0.0,0.0,0.0,0.0


In [13]:
df1 = df_AADevCatwParam[df_AADevCatwParam.isna().any(axis=1)]
display(df1)

df2 = df_AADevCatwParam[(df_AADevCatwParam == np.inf).any(axis=1)]
display (df2)

,AreaID,ClassID,county_id,DevCategory,parcel_id,job_spaces,resunits,Acres,sf_res,sf_com,acres_res,acres_com,ClassDescription,SFSplitRes,SFSplitCom,CapacityRes_DUA,CapacityCom_FAR,SFperHH,SFperEmp,PercentOpenSpace,RedevValuePerAcreRes,RedevValuePerAcreCom,RedevAndOr,RedevBldgAgeRes_Low,RedevBldgAgeRes_High,RedevBldgAgeCom_Low,RedevBldgAgeCom_High,RedevProb,SFRedevFullAdd,ClassOrder,AcresOpenSpaceRemoved,acres_undevelopable,acres_remain,acres_redevelopable,acres_developable,SFcom_undevelopable,SFres_undevelopable,SFcom_remain,SFres_remain,SFcom_redeveloped_orig,SFres_redeveloped_orig,SFcom_redeveloped_new,SFres_redeveloped_new,SFcom_developed,SFres_developed


,AreaID,ClassID,county_id,DevCategory,parcel_id,job_spaces,resunits,Acres,sf_res,sf_com,acres_res,acres_com,ClassDescription,SFSplitRes,SFSplitCom,CapacityRes_DUA,CapacityCom_FAR,SFperHH,SFperEmp,PercentOpenSpace,RedevValuePerAcreRes,RedevValuePerAcreCom,RedevAndOr,RedevBldgAgeRes_Low,RedevBldgAgeRes_High,RedevBldgAgeCom_Low,RedevBldgAgeCom_High,RedevProb,SFRedevFullAdd,ClassOrder,AcresOpenSpaceRemoved,acres_undevelopable,acres_remain,acres_redevelopable,acres_developable,SFcom_undevelopable,SFres_undevelopable,SFcom_remain,SFres_remain,SFcom_redeveloped_orig,SFres_redeveloped_orig,SFcom_redeveloped_new,SFres_redeveloped_new,SFcom_developed,SFres_developed


In [14]:
#calculate jobs and HH

#initialize
df_AADevCatwParam['job_spaces_undevelopable'   ] = 0.0
df_AADevCatwParam['job_spaces_remain'          ] = 0.0
df_AADevCatwParam['job_spaces_redeveloped_orig'] = 0.0
df_AADevCatwParam['job_spaces_redeveloped_new' ] = 0.0
df_AADevCatwParam['job_spaces_developed'       ] = 0.0

df_AADevCatwParam['resunits_undevelopable'     ] = 0.0
df_AADevCatwParam['resunits_remain'            ] = 0.0
df_AADevCatwParam['resunits_redeveloped_orig'  ] = 0.0
df_AADevCatwParam['resunits_redeveloped_new'   ] = 0.0
df_AADevCatwParam['resunits_developed'         ] = 0.0

#set undevelopable sf
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Undevelopable'), 'job_spaces_undevelopable'   ] =  df_AADevCatwParam['job_spaces'           ]
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Remain'       ), 'job_spaces_remain'          ] =  df_AADevCatwParam['job_spaces'           ]
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Redevelopable'), 'job_spaces_redeveloped_orig'] =  df_AADevCatwParam['job_spaces'           ]
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Redevelopable'), 'job_spaces_redeveloped_new' ] = (df_AADevCatwParam['SFcom_redeveloped_new'] / df_AADevCatwParam['SFperEmp']).astype('int64')
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Developable'  ), 'job_spaces_developed'       ] = (df_AADevCatwParam['SFcom_developed'      ] / df_AADevCatwParam['SFperEmp']).astype('int64')

df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Undevelopable'), 'resunits_undevelopable'     ] =  df_AADevCatwParam['resunits'             ]
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Remain'       ), 'resunits_remain'            ] =  df_AADevCatwParam['resunits'             ]
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Redevelopable'), 'resunits_redeveloped_orig'  ] =  df_AADevCatwParam['resunits'             ]
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Redevelopable'), 'resunits_redeveloped_new'   ] = (df_AADevCatwParam['SFres_redeveloped_new'] / df_AADevCatwParam['SFperHH' ]).astype('int64')
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Developable'  ), 'resunits_developed'         ] = (df_AADevCatwParam['SFres_developed'      ] / df_AADevCatwParam['SFperHH' ]).astype('int64')


df_AADevCatwParam

,AreaID,ClassID,county_id,DevCategory,parcel_id,job_spaces,resunits,Acres,sf_res,sf_com,acres_res,acres_com,ClassDescription,SFSplitRes,SFSplitCom,CapacityRes_DUA,CapacityCom_FAR,SFperHH,SFperEmp,PercentOpenSpace,RedevValuePerAcreRes,RedevValuePerAcreCom,RedevAndOr,RedevBldgAgeRes_Low,RedevBldgAgeRes_High,RedevBldgAgeCom_Low,RedevBldgAgeCom_High,RedevProb,SFRedevFullAdd,ClassOrder,AcresOpenSpaceRemoved,acres_undevelopable,acres_remain,acres_redevelopable,acres_developable,SFcom_undevelopable,SFres_undevelopable,SFcom_remain,SFres_remain,SFcom_redeveloped_orig,SFres_redeveloped_orig,SFcom_redeveloped_new,SFres_redeveloped_new,SFcom_developed,SFres_developed,job_spaces_undevelopable,job_spaces_remain,job_spaces_redeveloped_orig,job_spaces_redeveloped_new,job_spaces_developed,resunits_undevelopable,resunits_remain,resunits_redeveloped_orig,resunits_redeveloped_new,resunits_developed
0,0,a1,0.0,Developable,18.0,0.000000,0.000000,15.567788,0.000000,0.000000,0.000000,0.000000,Single Family A1,1.0,0.0,1.5,0.5,4000,350,0.15,40000,1000000,AND,50,300,20,300,1,Yes,1,13.232620,0.000000,0.000000,0.000000,15.567788,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,79395.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,19.0
1,0,a1,0.0,Undevelopable,8.0,0.000000,0.000000,14.733847,0.000000,0.000000,0.000000,0.000000,Single Family A1,1.0,0.0,1.5,0.5,4000,350,0.15,40000,1000000,AND,50,300,20,300,1,Yes,1,12.523770,14.733847,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
2,0,a1,1.0,Undevelopable,1.0,0.000000,0.000000,0.108287,0.000000,0.000000,0.000000,0.000000,Single Family A1,1.0,0.0,1.5,0.5,4000,350,0.15,40000,1000000,AND,50,300,20,300,1,Yes,1,0.092044,0.108287,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
3,0,a1,2.0,Developable,102.0,0.000000,0.000000,234.281314,0.000000,0.000000,0.000000,0.000000,Single Family A1,1.0,0.0,1.5,0.5,4000,350,0.15,40000,1000000,AND,50,300,20,300,1,Yes,1,199.139117,0.000000,0.000000,0.000000,234.281314,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,1194834.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,298.0
4,0,a1,2.0,Redevelopable,10.0,0.043236,8.002301,84.596637,10868.498573,42.155347,83.663641,0.932996,Single Family A1,1.0,0.0,1.5,0.5,4000,350,0.15,40000,1000000,AND,50,300,20,300,1,Yes,1,71.907142,0.000000,0.000000,84.596637,0.000000,0.0,0.0,0.000000,0.000000,42.155347,10868.498573,0.0,431442.0,0.0,0.0,0.0,0.000000,0.043236,0.0,0.0,0.0,0.000000,8.002301,107.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2549,79,g2,1.0,Remain,2.0,0.000005,0.000000,0.004237,0.000000,0.003396,0.000000,0.004237,Mixed-Use Suburban,0.7,0.3,30.0,1.0,1500,300,0.10,1500000,1200000,AND,60,120,20,120,1,No,31,0.003813,0.000000,0.004237,0.000000,0.000000,0.0,0.0,0.003396,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000005,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
2550,79,g2,1.0,Undevelopable,1.0,0.000000,0.000000,0.197377,0.000000,0.000000,0.000000,0.000000,Mixed-Use Suburban,0.7,0.3,30.0,1.0,1500,300,0.10,1500000,1200000,AND,60,120,20,120,1,No,31,0.177639,0.197377,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
2551,79,g3,1.0,Developable,28.0,0.000000,0.000000,80.238874,0.000000,0.000000,0.000000,0.000000,Mixed-Use City/TOD,0.5,0.5,60.0,4.0,1500,250,0.10,3000000,2400000,AND,60,120,20,120,1,No,32,72.214986,0.000000,0.000000,0.000000,80.238874,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,4285674.0,4285674.0,0.0,0.000000,0.000000,0.0,17142.0,0.0,0.000000,0.000000,0.0,2857.0
2552,79,g3,1.0,Redevelopable,1.0,0.000000,0.

In [15]:
df_AADevCatwParam[df_AADevCatwParam['AreaID']==52]

,AreaID,ClassID,county_id,DevCategory,parcel_id,job_spaces,resunits,Acres,sf_res,sf_com,acres_res,acres_com,ClassDescription,SFSplitRes,SFSplitCom,CapacityRes_DUA,CapacityCom_FAR,SFperHH,SFperEmp,PercentOpenSpace,RedevValuePerAcreRes,RedevValuePerAcreCom,RedevAndOr,RedevBldgAgeRes_Low,RedevBldgAgeRes_High,RedevBldgAgeCom_Low,RedevBldgAgeCom_High,RedevProb,SFRedevFullAdd,ClassOrder,AcresOpenSpaceRemoved,acres_undevelopable,acres_remain,acres_redevelopable,acres_developable,SFcom_undevelopable,SFres_undevelopable,SFcom_remain,SFres_remain,SFcom_redeveloped_orig,SFres_redeveloped_orig,SFcom_redeveloped_new,SFres_redeveloped_new,SFcom_developed,SFres_developed,job_spaces_undevelopable,job_spaces_remain,job_spaces_redeveloped_orig,job_spaces_redeveloped_new,job_spaces_developed,resunits_undevelopable,resunits_remain,resunits_redeveloped_orig,resunits_redeveloped_new,resunits_developed
1681,52,b2,3.0,Developable,9.0,0.000000e+00,0.000000,0.345956,0.000000e+00,0.000000e+00,0.000000,0.000000,Single Family B2,1.00,0.00,2.5,0.5,3500,350,0.15,125000,1000000,AND,50,300,20,300,1,Yes,7,0.294063,0.000000,0.000000,0.000000,0.345956,0.000000,0.000000,0.000000e+00,0.000000e+00,0.0,0.000000,0.0,0.0,0.0,2573.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
1682,52,b2,3.0,Redevelopable,6.0,0.000000e+00,5.394264,6.490133,6.527636e+03,0.000000e+00,6.490133,0.000000,Single Family B2,1.00,0.00,2.5,0.5,3500,350,0.15,125000,1000000,AND,50,300,20,300,1,Yes,7,5.516613,0.000000,0.000000,6.490133,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.0,6527.636278,0.0,48270.0,0.0,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.000000,0.000000,5.394264,13.0,0.0
1683,52,b2,3.0,Remain,59.0,1.416127e-12,63.208106,49.133030,8.867787e+04,8.964082e-10,49.133021,0.000009,Single Family B2,1.00,0.00,2.5,0.5,3500,350,0.15,125000,1000000,AND,50,300,20,300,1,Yes,7,41.763075,0.000000,49.133030,0.000000,0.000000,0.000000,0.000000,8.964082e-10,8.867787e+04,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,1.416127e-12,0.0,0.0,0.0,0.000000,63.208106,0.000000,0.0,0.0
1684,52,b2,3.0,Undevelopable,3.0,0.000000e+00,0.000000,0.112441,0.000000e+00,0.000000e+00,0.000000,0.000000,Single Family B2,1.00,0.00,2.5,0.5,3500,350,0.15,125000,1000000,AND,50,300,20,300,1,Yes,7,0.095575,0.112441,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
1685,52,b3,3.0,Developable,28.0,0.000000e+00,0.000000,24.563466,0.000000e+00,0.000000e+00,0.000000,0.000000,Single Family B3,1.00,0.00,2.5,0.5,3500,350,0.15,150000,1000000,AND,50,300,20,300,1,Yes,8,20.878946,0.000000,0.000000,0.000000,24.563466,0.000000,0.000000,0.000000e+00,0.000000e+00,0.0,0.000000,0.0,0.0,0.0,182690.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,52.0
1686,52,b3,3.0,Redevelopable,16.0,0.000000e+00,11.929072,16.816116,1.786910e+04,0.000000e+00,16.816116,0.000000,Single Family B3,1.00,0.00,2.5,0.5,3500,350,0.15,150000,1000000,AND,50,300,20,300,1,Yes,8,14.293699,0.000000,0.000000,16.816116,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.0,17869.100066,0.0,125069.0,0.0,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.000000,0.000000,11.929072,35.0,0.0
1687,52,b3,3.0,Remain,483.0,2.698530e+02,409.644171,209.205270,6.947410e+05,1.814376e+05,196.368798,12.836472,Single Family B3,1.00,0.00,2.5,0.5,3500,350,0.15,150000,1000000,AND,50,300,20,300,1,Yes,8,177.824480,0.000000,209.205270,0.000000,0.000000,0.000000,0.000000,1.814376e+05,6.947410e+05,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,2.698530e+02,0.0,0.0,0.0,0.000000,409.644171,0.000000,0.0,0.0
1688,52,b3,3.0,Undevelopable,59.0,0.000000e+00,10.570028,14.972563,1.329240e+04,0.000000e+00,0.000000,0.000000,Single Family B3,1.00,0.00,2.5,0.5,3500,350,0.15,150000,1000000,AND,50,300,20,300,1,Yes,8,12.726679,14.972563,0.000000,0.000000,0.000000,0.000000,13292.396409,0.000000e+00,0.000000e+00,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,10.570028,0

In [16]:
#calculate totals

df_AASummary = df_AADevCatwParam.groupby(['county_id','AreaID','ClassID','ClassDescription','ClassOrder'], as_index=False).agg({"DevCategory":[np.size], "parcel_id":[np.sum], "Acres":[np.sum], "acres_undevelopable":[np.sum], "acres_remain":[np.sum], "acres_redevelopable":[np.sum], "acres_developable":[np.sum], "SFcom_undevelopable":[np.sum], "SFres_undevelopable":[np.sum], "SFcom_remain":[np.sum], "SFres_remain":[np.sum], "SFcom_redeveloped_orig":[np.sum], "SFres_redeveloped_orig":[np.sum], "SFcom_redeveloped_new":[np.sum], "SFres_redeveloped_new":[np.sum], "SFcom_developed":[np.sum], "SFres_developed":[np.sum], "job_spaces_undevelopable":[np.sum], "job_spaces_remain":[np.sum], "job_spaces_redeveloped_orig":[np.sum], "job_spaces_redeveloped_new":[np.sum], "job_spaces_developed":[np.sum], "resunits_undevelopable":[np.sum], "resunits_remain":[np.sum], "resunits_redeveloped_orig":[np.sum], "resunits_redeveloped_new":[np.sum], "resunits_developed":[np.sum]})
df_AASummary.columns = df_AASummary.columns.droplevel(1)

df_AASummaryTotals = df_AASummary.copy()

df_AASummaryTotals['job_spaces_orig'  ] = (df_AASummaryTotals['job_spaces_undevelopable'] + df_AASummaryTotals['job_spaces_remain'] + df_AASummaryTotals['job_spaces_redeveloped_orig']).astype('int64')
df_AASummaryTotals['job_spaces_new'   ] = (df_AASummaryTotals['job_spaces_undevelopable'] + df_AASummaryTotals['job_spaces_remain'] + df_AASummaryTotals['job_spaces_redeveloped_new' ] + df_AASummaryTotals['job_spaces_developed']).astype('int64')
df_AASummaryTotals['job_spaces_change'] = (df_AASummaryTotals['job_spaces_new'          ] - df_AASummaryTotals['job_spaces_orig'  ]).round(0)

df_AASummaryTotals['resunits_orig'    ] = (df_AASummaryTotals['resunits_undevelopable'  ] + df_AASummaryTotals['resunits_remain'  ] + df_AASummaryTotals['resunits_redeveloped_orig'  ]).astype('int64')
df_AASummaryTotals['resunits_new'     ] = (df_AASummaryTotals['resunits_undevelopable'  ] + df_AASummaryTotals['resunits_remain'  ] + df_AASummaryTotals['resunits_redeveloped_new'   ] + df_AASummaryTotals['resunits_developed'  ]).astype('int64')
df_AASummaryTotals['resunits_change'  ] = (df_AASummaryTotals['resunits_new'            ] - df_AASummaryTotals['resunits_orig'    ]).round(0)

df_AASummaryTotals = df_AASummaryTotals[['county_id','AreaID','ClassID','ClassDescription','ClassOrder','parcel_id','Acres','acres_redevelopable','acres_developable','job_spaces_orig','job_spaces_new','job_spaces_change','resunits_orig','resunits_new','resunits_change','job_spaces_undevelopable','job_spaces_remain','job_spaces_redeveloped_orig','job_spaces_redeveloped_new','job_spaces_developed','resunits_undevelopable','resunits_remain','resunits_redeveloped_orig','resunits_redeveloped_new','resunits_developed']]

display(df_AASummaryTotals.sum())

df_AASummaryTotals['density_hhemp_orig'  ] = ((df_AASummaryTotals['job_spaces_orig'  ] + df_AASummaryTotals['resunits_orig'     ]) / df_AASummaryTotals['Acres']).round(1)
df_AASummaryTotals['density_hhemp_new'   ] = ((df_AASummaryTotals['job_spaces_new'   ] + df_AASummaryTotals['resunits_new'      ]) / df_AASummaryTotals['Acres']).round(1)
df_AASummaryTotals['density_hhemp_change'] =  (df_AASummaryTotals['density_hhemp_new'] - df_AASummaryTotals['density_hhemp_orig']).round(1)


county_id                                                                   1880
AreaID                                                                     29770
ClassID                        a1a3a4a5c1a2a4b4b4a1a4d3a1a4b4b5b1c1d1b3a1c3a2...
ClassDescription               Single Family A1Single Family A3Single Family ...
ClassOrder                                                                 12741
parcel_id                                                                 982966
Acres                                                                     333944
acres_redevelopable                                                      17002.8
acres_developable                                                         100391
job_spaces_orig                                                           346007
job_spaces_new                                                           3890286
job_spaces_change                                                        3544279
resunits_orig               

In [17]:
df_AASummaryTotals
#export to csv
df_AASummaryTotals.to_csv(os.path.join(dir_results, r'SummaryTable_AreaClass_wTotals.csv'))

In [18]:
##create map centered on Salt Lake
#map_parcels_areas_den = gis.map('Salt Lake')
#
#sdf_AADevCatwParam = pd.DataFrame.merge(sdf_AAid, df_AASummaryTotals, on=['AreaID','ClassID'], how='left')
#
##expression to classify employment growth
#arcade_expression_denchange = ("var v = $feature.density_hhemp_change;"
#                                     "if      (v<0    ) { return 'class0'; }"
#                                     "if      (v<10   ) { return 'class1'; }"
#                                     "else if (v<25   ) { return 'class2'; }"
#                                     "else if (v<50   ) { return 'class3'; }"
#                                     "else              { return 'class4'; }")
#
##symbology for enrollment classes
#uv_denchange = [
#                #{"value":"class0", "label":"Density Decline", "symbol":{"type":"esriSFS","color":[  0,  0,  0,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
#                {"value":"class1", "label":"0 to 10"      , "symbol":{"type":"esriSFS","color":[130,165,217,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
#                {"value":"class2", "label":"10 to 25"     , "symbol":{"type":"esriSFS","color":[140,125,164,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
#                {"value":"class3", "label":"25 to 50"     , "symbol":{"type":"esriSFS","color":[149, 85,111,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
#                {"value":"class4", "label":"More than 50" , "symbol":{"type":"esriSFS","color":[168,  4,  4,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}}
#               ]
#
#
##define sdf layer
#sdf_AADevCatwParam.spatial.plot(map_widget = map_parcels_areas_den,
#                                renderer_type='u-a', #'u-a' stands for uniqe value with arcade expression
#                                unique_values=uv_denchange,
#                                arcade_expression=arcade_expression_denchange,
#                                default_symbol="" #don't include an 'other' category
#                                )
#
##define map characteristics
#map_parcels_areas_den.layout.height='500px'
#map_parcels_areas_den.legend=True
#
##map title
#display(Markdown('<h2><center>Density Change by Analysis Area</center></h2>'))
#
##display map
#map_parcels_areas_den

# Area Summary

In [19]:
df_AreaSummary = df_AASummaryTotals.groupby(['AreaID'], as_index=False).agg({"Acres":[np.sum],"job_spaces_orig":[np.sum],"job_spaces_new":[np.sum],"job_spaces_change":[np.sum],"resunits_orig":[np.sum],"resunits_new":[np.sum],"resunits_change":[np.sum],"resunits_undevelopable":[np.sum],"resunits_remain":[np.sum],"resunits_redeveloped_orig":[np.sum],"resunits_redeveloped_new":[np.sum],"resunits_developed":[np.sum]})
df_AreaSummary.columns = df_AreaSummary.columns.droplevel(1)

#read in csv with area names
df_AreaNames = pd.read_csv(AnalysisAreasTable)

#merge area names with area summary
df_AreaSummary_wNames = pd.DataFrame.merge(df_AreaNames, df_AreaSummary, on="AreaID")

#add total line to area summary with names
df_AreaSummary_wNames_wTotals = df_AreaSummary_wNames.append(df_AreaSummary_wNames.sum(numeric_only=True).rename('Total'))

#recalculate densities, round to nearest whole number (calculate before rounding of jobs/units)
df_AreaSummary_wNames_wTotals['density_hhemp_orig'  ] = ((df_AreaSummary_wNames_wTotals['job_spaces_orig'  ] + df_AreaSummary_wNames_wTotals['resunits_orig'     ]) / df_AreaSummary_wNames_wTotals['Acres']).round(0)
df_AreaSummary_wNames_wTotals['density_hhemp_new'   ] = ((df_AreaSummary_wNames_wTotals['job_spaces_new'   ] + df_AreaSummary_wNames_wTotals['resunits_new'      ]) / df_AreaSummary_wNames_wTotals['Acres']).round(0)
df_AreaSummary_wNames_wTotals['density_hhemp_change'] =  (df_AreaSummary_wNames_wTotals['density_hhemp_new'] - df_AreaSummary_wNames_wTotals['density_hhemp_orig']).round(0)

#recalculate based on rounded values, rounded to nearest hundred
df_AreaSummary_wNames_wTotals['job_spaces_orig'     ] = (df_AreaSummary_wNames_wTotals['job_spaces_orig'   ]).round(-2)
df_AreaSummary_wNames_wTotals['job_spaces_new'      ] = (df_AreaSummary_wNames_wTotals['job_spaces_new'    ]).round(-2)
df_AreaSummary_wNames_wTotals['job_spaces_change'   ] =  df_AreaSummary_wNames_wTotals['job_spaces_new'    ] - df_AreaSummary_wNames_wTotals['job_spaces_orig'] 

df_AreaSummary_wNames_wTotals['resunits_orig'       ] = (df_AreaSummary_wNames_wTotals['resunits_orig'     ]).round(-2)
df_AreaSummary_wNames_wTotals['resunits_new'        ] = (df_AreaSummary_wNames_wTotals['resunits_new'      ]).round(-2)
df_AreaSummary_wNames_wTotals['resunits_change'     ] =  df_AreaSummary_wNames_wTotals['resunits_new'      ] - df_AreaSummary_wNames_wTotals['resunits_orig'] 

#change index to Area ID
df_AreaSummary_wNames_wTotals = df_AreaSummary_wNames_wTotals.set_index(['AreaID'])

#display table with numeric styling
display(df_AreaSummary_wNames_wTotals.style.format({"Acres":"{:,.1f}","job_spaces_orig":"{:,.0f}","job_spaces_new":"{:,.0f}","job_spaces_change":"{:,.0f}","resunits_orig":"{:,.0f}","resunits_new":"{:,.0f}","resunits_change":"{:,.0f}","density_hhemp_orig":"{:,.0f}","density_hhemp_new":"{:,.0f}","density_hhemp_change":"{:,.0f}","resunits_undevelopable":"{:,.0f}","resunits_remain":"{:,.0f}","resunits_redeveloped_orig":"{:,.0f}","resunits_redeveloped_new":"{:,.0f}","resunits_developed":"{:,.0f}"}))

#export to csv
df_AreaSummary_wNames_wTotals.to_csv(os.path.join(dir_results, r'SummaryTable_Area_wTotals.csv'))

,AnalysisAreaName,Acres,job_spaces_orig,job_spaces_new,job_spaces_change,resunits_orig,resunits_new,resunits_change,resunits_undevelopable,resunits_remain,resunits_redeveloped_orig,resunits_redeveloped_new,resunits_developed,density_hhemp_orig,density_hhemp_new,density_hhemp_change
AreaID,,,,,,,,,,,,,,,,
0.0,nan,"31,290.9","2,400","10,100","7,700","6,400","17,800","11,400","1,850","4,088",488,"1,906","10,014",0,1,1
1.0,Alpine,"3,803.8",600,600,0,"2,600","4,400","1,800",7,"2,583",24,81,"1,712",1,1,0
3.0,American Fork,"4,265.5","1,100","3,700","2,600","8,100","15,600","7,500",126,"7,690",319,"1,015","6,822",2,5,3
4.0,Bluffdale,"4,153.7",100,"63,400","63,300","2,900","15,300","12,400",602,"2,313",24,754,"11,663",1,19,18
5.0,Bountiful,"6,180.7","6,900","15,100","8,200","15,500","19,500","4,000",212,"14,908",370,"1,861","2,542",4,6,2
7.0,Cedar Fort,453.1,0,0,0,100,500,400,0,104,4,45,356,0,1,1
8.0,Cedar Hills,"1,035.8",100,200,100,"2,500","2,900",400,1,"2,533",5,6,353,3,3,0
9.0,Centerville,"1,345.5",500,"7,800","7,300","4,800","7,800","3,000",61,"4,684",98,347,"2,726",4,12,8
10.0,Clearfield,"2,128.9","4,300","150,100","145,800","9,000","28,400","19,400",134,"8,040",791,"8,276","11,927",6,84,78


# Class Summary

In [20]:
#Class table is aggregate of df_AASummaryTotals
df_ClassSummaryTable = df_AASummaryTotals.groupby(['ClassOrder','ClassDescription'], as_index=False).agg({"Acres":[np.sum],"acres_redevelopable":[np.sum],"acres_developable":[np.sum],"job_spaces_orig":[np.sum],"job_spaces_new":[np.sum],"job_spaces_change":[np.sum],"resunits_orig":[np.sum],"resunits_new":[np.sum],"resunits_change":[np.sum], "job_spaces_undevelopable":[np.sum], "job_spaces_remain":[np.sum], "job_spaces_redeveloped_orig":[np.sum], "job_spaces_redeveloped_new":[np.sum], "job_spaces_developed":[np.sum], "resunits_undevelopable":[np.sum], "resunits_remain":[np.sum], "resunits_redeveloped_orig":[np.sum], "resunits_redeveloped_new":[np.sum], "resunits_developed":[np.sum]})

#drop aggregate description from column name (sum, etc)
df_ClassSummaryTable.columns = df_ClassSummaryTable.columns.droplevel(1)

#add totals row
df_ClassSummaryTable_wTotals = df_ClassSummaryTable.append(df_ClassSummaryTable.sum(numeric_only=True).rename('Total'))

#recalculate densities - calculate densities before rounding
df_ClassSummaryTable_wTotals['density_hhemp_orig'  ] = ((df_ClassSummaryTable_wTotals['job_spaces_orig'  ] + df_ClassSummaryTable_wTotals['resunits_orig'     ]) / df_ClassSummaryTable_wTotals['Acres']).round(0)
df_ClassSummaryTable_wTotals['density_hhemp_new'   ] = ((df_ClassSummaryTable_wTotals['job_spaces_new'   ] + df_ClassSummaryTable_wTotals['resunits_new'      ]) / df_ClassSummaryTable_wTotals['Acres']).round(0)
df_ClassSummaryTable_wTotals['density_hhemp_change'] =  (df_ClassSummaryTable_wTotals['density_hhemp_new'] - df_ClassSummaryTable_wTotals['density_hhemp_orig']).round(0)

#calculate rounded values
df_ClassSummaryTable_wTotals['job_spaces_orig'     ] =  (df_ClassSummaryTable_wTotals['job_spaces_orig'  ]).round(-2)
df_ClassSummaryTable_wTotals['job_spaces_new'      ] =  (df_ClassSummaryTable_wTotals['job_spaces_new'   ]).round(-2)
df_ClassSummaryTable_wTotals['job_spaces_change'   ] =   df_ClassSummaryTable_wTotals['job_spaces_new'   ] - df_ClassSummaryTable_wTotals['job_spaces_orig'] 

df_ClassSummaryTable_wTotals['resunits_orig'       ] =  (df_ClassSummaryTable_wTotals['resunits_orig'    ]).round(-2)
df_ClassSummaryTable_wTotals['resunits_new'        ] =  (df_ClassSummaryTable_wTotals['resunits_new'     ]).round(-2)
df_ClassSummaryTable_wTotals['resunits_change'     ] =   df_ClassSummaryTable_wTotals['resunits_new'     ] - df_ClassSummaryTable_wTotals['resunits_orig'] 

#fix last row name to be (HARD CODED CLASS ORDER-FIX!!)
df_ClassSummaryTable_wTotals.loc[(df_ClassSummaryTable_wTotals['ClassOrder'] == 666), 'ClassDescription'] = 'Total'
df_ClassSummaryTable_wTotals.loc[(df_ClassSummaryTable_wTotals['ClassOrder'] == 666), 'ClassOrder'] = 37

#set index to class order
df_ClassSummaryTable_wTotals = df_ClassSummaryTable_wTotals.set_index(['ClassOrder'])

df_ClassSummaryTable_wTotals['job_spaces_redev_add'       ] = df_ClassSummaryTable_wTotals['job_spaces_redeveloped_new'] -  df_ClassSummaryTable_wTotals['job_spaces_redeveloped_orig']
df_ClassSummaryTable_wTotals['resunits_redev_add'         ] = df_ClassSummaryTable_wTotals['resunits_redeveloped_new'  ] -  df_ClassSummaryTable_wTotals['resunits_redeveloped_orig'  ]

df_ClassSummaryTable_wTotals['job_spaces_redev_percentadd'] = df_ClassSummaryTable_wTotals['job_spaces_redev_add'      ] / (df_ClassSummaryTable_wTotals['job_spaces_redev_add'       ] + df_ClassSummaryTable_wTotals['job_spaces_developed'])
df_ClassSummaryTable_wTotals['job_spaces_dev_percentadd'  ] = df_ClassSummaryTable_wTotals['job_spaces_developed'      ] / (df_ClassSummaryTable_wTotals['job_spaces_redev_add'       ] + df_ClassSummaryTable_wTotals['job_spaces_developed'])

df_ClassSummaryTable_wTotals['resunits_redev_percentadd'  ] = df_ClassSummaryTable_wTotals['resunits_redev_add'        ] / (df_ClassSummaryTable_wTotals['resunits_redev_add'         ] + df_ClassSummaryTable_wTotals['resunits_developed'  ])
df_ClassSummaryTable_wTotals['resunits_dev_percentadd'    ] = df_ClassSummaryTable_wTotals['resunits_developed'        ] / (df_ClassSummaryTable_wTotals['resunits_redev_add'         ] + df_ClassSummaryTable_wTotals['resunits_developed'  ])

df_ClassSummaryTable_wTotals['Acres_Redev_Percent'        ] =  df_ClassSummaryTable_wTotals['acres_redevelopable'      ]                                                                / df_ClassSummaryTable_wTotals['Acres']
df_ClassSummaryTable_wTotals['Acres_Dev_Percent'          ] =  df_ClassSummaryTable_wTotals['acres_developable'        ]                                                                / df_ClassSummaryTable_wTotals['Acres']

df_ClassSummaryTable_wTotals['Acres_Unchanged_Percent'    ] = 1 - df_ClassSummaryTable_wTotals['Acres_Dev_Percent'] - df_ClassSummaryTable_wTotals['Acres_Redev_Percent']

df_ClassSummaryTable_wTotals = df_ClassSummaryTable_wTotals[['ClassDescription','Acres','job_spaces_orig','job_spaces_new','job_spaces_change','resunits_orig','resunits_new','resunits_change','density_hhemp_orig','density_hhemp_new','density_hhemp_change','Acres_Unchanged_Percent','Acres_Redev_Percent','Acres_Dev_Percent','job_spaces_redev_percentadd','job_spaces_dev_percentadd','resunits_redev_percentadd','resunits_dev_percentadd','resunits_undevelopable','resunits_remain','resunits_redeveloped_orig','resunits_redeveloped_new','resunits_developed']]

#display with formatted numbers
display(df_ClassSummaryTable_wTotals.style.format({"Acres":"{:,.0f}","job_spaces_orig":"{:,.0f}","job_spaces_new":"{:,.0f}","job_spaces_change":"{:,.0f}","resunits_orig":"{:,.0f}","resunits_new":"{:,.0f}","resunits_change":"{:,.0f}","density_hhemp_orig":"{:,.0f}","density_hhemp_new":"{:,.0f}","density_hhemp_change":"{:,.0f}","Acres_Unchanged_Percent":"{:,.0%}","Acres_Redev_Percent":"{:,.0%}","Acres_Dev_Percent":"{:,.0%}","job_spaces_redev_percentadd":"{:,.0%}","job_spaces_dev_percentadd":"{:,.0%}","resunits_redev_percentadd":"{:,.0%}","resunits_dev_percentadd":"{:,.0%}","resunits_undevelopable":"{:,.0f}","resunits_remain":"{:,.0f}","resunits_redeveloped_orig":"{:,.0f}","resunits_redeveloped_new":"{:,.0f}","resunits_developed":"{:,.0f}"}))

#export to csv
df_ClassSummaryTable_wTotals.to_csv(os.path.join(dir_results, r'SummaryTable_Class_wTotals.csv'))

,ClassDescription,Acres,job_spaces_orig,job_spaces_new,job_spaces_change,resunits_orig,resunits_new,resunits_change,density_hhemp_orig,density_hhemp_new,density_hhemp_change,Acres_Unchanged_Percent,Acres_Redev_Percent,Acres_Dev_Percent,job_spaces_redev_percentadd,job_spaces_dev_percentadd,resunits_redev_percentadd,resunits_dev_percentadd,resunits_undevelopable,resunits_remain,resunits_redeveloped_orig,resunits_redeveloped_new,resunits_developed
ClassOrder,,,,,,,,,,,,,,,,,,,,,,,
1.0,Single Family A1,"18,538",200,200,0,"1,400","14,300","12,900",0,1,1,45%,4%,51%,100%,-0%,6%,94%,95,"1,264",65,830,"12,133"
2.0,Single Family A2,"15,437",500,500,0,"3,200","9,200","6,000",0,1,1,69%,2%,29%,100%,-0%,5%,95%,"1,172","1,970",66,360,"5,694"
3.0,Single Family A3,"19,111","1,500","1,400",-100,"5,600","9,700","4,100",0,1,1,83%,2%,15%,100%,-0%,9%,91%,469,"5,049",53,434,"3,715"
4.0,Single Family A4,"8,931","1,000",900,-100,"3,600","7,300","3,700",1,1,0,68%,2%,30%,100%,-0%,4%,96%,32,"3,591",27,190,"3,457"
5.0,Single Family A5,"5,900","1,000",900,-100,"2,800","4,600","1,800",1,1,0,75%,3%,22%,100%,-0%,9%,91%,106,"2,653",71,224,"1,623"
6.0,Single Family B1,"10,482","1,100","1,100",0,"2,900","15,800","12,900",0,2,2,42%,3%,55%,100%,-0%,4%,96%,71,"2,839",45,586,"12,350"
7.0,Single Family B2,"14,800",900,800,-100,"12,600","26,100","13,500",1,2,1,57%,4%,39%,100%,-0%,8%,92%,65,"12,426",164,"1,238","12,365"
8.0,Single Family B3,"9,017","2,600","2,500",-100,"11,500","17,300","5,800",2,2,0,69%,3%,28%,100%,-0%,7%,93%,42,"11,395",123,511,"5,365"
9.0,Single Family B4,"9,693","1,700","1,600",-100,"10,600","16,200","5,600",1,2,1,72%,3%,25%,100%,-0%,8%,92%,99,"10,261",238,699,"5,140"


In [21]:
#df_AASummaryTotals
df_ClassSummaryTable_wTotals

,ClassDescription,Acres,job_spaces_orig,job_spaces_new,job_spaces_change,resunits_orig,resunits_new,resunits_change,density_hhemp_orig,density_hhemp_new,density_hhemp_change,Acres_Unchanged_Percent,Acres_Redev_Percent,Acres_Dev_Percent,job_spaces_redev_percentadd,job_spaces_dev_percentadd,resunits_redev_percentadd,resunits_dev_percentadd,resunits_undevelopable,resunits_remain,resunits_redeveloped_orig,resunits_redeveloped_new,resunits_developed
ClassOrder,,,,,,,,,,,,,,,,,,,,,,,
1.0,Single Family A1,18538.347701,200.0,200.0,0.0,1400.0,14300.0,12900.0,0.0,1.0,1.0,0.450866,0.035368,0.513766,1.000000,-0.000000,0.059313,0.940687,95.306495,1264.379551,64.982834,830.0,12133.0
2.0,Single Family A2,15436.837229,500.0,500.0,0.0,3200.0,9200.0,6000.0,0.0,1.0,1.0,0.691460,0.018635,0.289905,1.000000,-0.000000,0.049061,0.950939,1172.109517,1970.264193,66.234551,360.0,5694.0
3.0,Single Family A3,19110.634366,1500.0,1400.0,-100.0,5600.0,9700.0,4100.0,0.0,1.0,1.0,0.829003,0.018026,0.152971,1.000000,-0.000000,0.092951,0.907049,468.929771,5049.180039,53.301451,434.0,3715.0
4.0,Single Family A4,8931.162388,1000.0,900.0,-100.0,3600.0,7300.0,3700.0,1.0,1.0,0.0,0.678499,0.017124,0.304377,1.000000,-0.000000,0.044916,0.955084,31.781984,3590.658076,27.423478,190.0,3457.0
5.0,Single Family A5,5900.436936,1000.0,900.0,-100.0,2800.0,4600.0,1800.0,1.0,1.0,0.0,0.753268,0.030260,0.216472,1.000000,-0.000000,0.086163,0.913837,105.879464,2653.222336,70.972310,224.0,1623.0
6.0,Single Family B1,10481.991425,1100.0,1100.0,0.0,2900.0,15800.0,12900.0,0.0,2.0,2.0,0.418598,0.026597,0.554805,1.000000,-0.000000,0.041997,0.958003,71.000495,2839.354080,44.599539,586.0,12350.0
7.0,Single Family B2,14799.932195,900.0,800.0,-100.0,12600.0,26100.0,13500.0,1.0,2.0,1.0,0.566880,0.039642,0.393478,1.000000,-0.000000,0.079900,0.920100,65.031326,12426.359475,164.236528,1238.0,12365.0
8.0,Single Family B3,9017.164312,2600.0,2500.0,-100.0,11500.0,17300.0,5800.0,2.0,2.0,0.0,0.692390,0.027029,0.280581,1.000000,-0.000000,0.067524,0.932476,42.457938,11394.559158,122.500548,511.0,5365.0
9.0,Single Family B4,9693.220105,1700.0,1600.0,-100.0,10600.0,16200.0,5600.0,1.0,2.0,1.0,0.715555,0.034279,0.250166,1.000000,-0.000000,0.082327,0.917673,98.980395,10260.575601,237.878475,699.0,5140.0


# Summary by County

In [22]:
df_CountySummary = df_AASummaryTotals.groupby(['county_id'], as_index=False).agg({"Acres":[np.sum],"job_spaces_orig":[np.sum],"job_spaces_new":[np.sum],"job_spaces_change":[np.sum],"resunits_orig":[np.sum],"resunits_new":[np.sum],"resunits_change":[np.sum],"resunits_undevelopable":[np.sum],"resunits_remain":[np.sum],"resunits_redeveloped_orig":[np.sum],"resunits_redeveloped_new":[np.sum],"resunits_developed":[np.sum]})
df_CountySummary.columns = df_CountySummary.columns.droplevel(1)

#read in csv with area names
df_CountyNames = pd.read_csv(CountiesTable)

#merge area names with area summary
df_CountySummary_wNames = pd.DataFrame.merge(df_CountyNames, df_CountySummary, on="county_id")

#add total line to area summary with names
df_CountySummary_wNames_wTotals = df_CountySummary_wNames.append(df_CountySummary_wNames.sum(numeric_only=True).rename('Total'))

#recalculate densities, round to nearest whole number (calculate before rounding of jobs/units)
df_CountySummary_wNames_wTotals['density_hhemp_orig'  ] = ((df_CountySummary_wNames_wTotals['job_spaces_orig'  ] + df_CountySummary_wNames_wTotals['resunits_orig'     ]) / df_CountySummary_wNames_wTotals['Acres']).round(0)
df_CountySummary_wNames_wTotals['density_hhemp_new'   ] = ((df_CountySummary_wNames_wTotals['job_spaces_new'   ] + df_CountySummary_wNames_wTotals['resunits_new'      ]) / df_CountySummary_wNames_wTotals['Acres']).round(0)
df_CountySummary_wNames_wTotals['density_hhemp_change'] =  (df_CountySummary_wNames_wTotals['density_hhemp_new'] - df_CountySummary_wNames_wTotals['density_hhemp_orig']).round(0)

#recalculate based on rounded values, rounded to nearest hundred
df_CountySummary_wNames_wTotals['job_spaces_orig'     ] = (df_CountySummary_wNames_wTotals['job_spaces_orig'   ]).round(-2)
df_CountySummary_wNames_wTotals['job_spaces_new'      ] = (df_CountySummary_wNames_wTotals['job_spaces_new'    ]).round(-2)
df_CountySummary_wNames_wTotals['job_spaces_change'   ] =  df_CountySummary_wNames_wTotals['job_spaces_new'    ] - df_CountySummary_wNames_wTotals['job_spaces_orig'] 

df_CountySummary_wNames_wTotals['resunits_orig'       ] = (df_CountySummary_wNames_wTotals['resunits_orig'     ]).round(-2)
df_CountySummary_wNames_wTotals['resunits_new'        ] = (df_CountySummary_wNames_wTotals['resunits_new'      ]).round(-2)
df_CountySummary_wNames_wTotals['resunits_change'     ] =  df_CountySummary_wNames_wTotals['resunits_new'      ] - df_CountySummary_wNames_wTotals['resunits_orig'] 

#change index to Area ID
df_CountySummary_wNames_wTotals = df_CountySummary_wNames_wTotals.set_index(['CO_ORDER'])

#display table with numeric styling
display(df_CountySummary_wNames_wTotals.style.format({"Acres":"{:,.1f}","job_spaces_orig":"{:,.0f}","job_spaces_new":"{:,.0f}","job_spaces_change":"{:,.0f}","resunits_orig":"{:,.0f}","resunits_new":"{:,.0f}","resunits_change":"{:,.0f}","density_hhemp_orig":"{:,.0f}","density_hhemp_new":"{:,.0f}","density_hhemp_change":"{:,.0f}","resunits_undevelopable":"{:,.0f}","resunits_remain":"{:,.0f}","resunits_redeveloped_orig":"{:,.0f}","resunits_redeveloped_new":"{:,.0f}","resunits_developed":"{:,.0f}"}))

#export to csv
df_CountySummary_wNames_wTotals.to_csv(os.path.join(dir_results, r'SummaryTable_County_wTotals.csv'))

,county_id,CO_NAME,Acres,job_spaces_orig,job_spaces_new,job_spaces_change,resunits_orig,resunits_new,resunits_change,resunits_undevelopable,resunits_remain,resunits_redeveloped_orig,resunits_redeveloped_new,resunits_developed,density_hhemp_orig,density_hhemp_new,density_hhemp_change
CO_ORDER,,,,,,,,,,,,,,,,,
1.0,3.000000,Weber,"64,781.8","41,900","408,200","366,300","89,600","178,600","89,000","2,700","83,063","3,902","21,096","71,867",2,9,7
2.0,1.000000,Davis,"47,045.7","42,600","769,600","727,000","99,000","233,100","134,100",710,"96,073","2,272","36,939","99,399",3,21,18
3.0,2.000000,Salt Lake,"101,583.2","204,900","2,302,700","2,097,800","373,400","745,800","372,400","4,414","348,929","20,164","173,586","219,000",6,30,24
4.0,4.000000,Utah,"118,991.2","56,600","409,700","353,100","153,000","428,300","275,300","1,981","145,539","5,524","18,952","261,970",2,7,5
10.0,10.000000,nan,"332,401.8","346,000","3,890,300","3,544,300","714,900","1,585,900","871,000","9,805","673,603","31,861","250,573","652,236",3,16,13


In [23]:
df_CountySummary_wNames_wTotals

,county_id,CO_NAME,Acres,job_spaces_orig,job_spaces_new,job_spaces_change,resunits_orig,resunits_new,resunits_change,resunits_undevelopable,resunits_remain,resunits_redeveloped_orig,resunits_redeveloped_new,resunits_developed,density_hhemp_orig,density_hhemp_new,density_hhemp_change
CO_ORDER,,,,,,,,,,,,,,,,,
1.0,3.0,Weber,64781.783824,41900.0,408200.0,366300.0,89600.0,178600.0,89000.0,2700.220286,83063.156389,3901.611544,21096.0,71867.0,2.0,9.0,7.0
2.0,1.0,Davis,47045.662860,42600.0,769600.0,727000.0,99000.0,233100.0,134100.0,710.016725,96072.517445,2271.606350,36939.0,99399.0,3.0,21.0,18.0
3.0,2.0,Salt Lake,101583.167376,204900.0,2302700.0,2097800.0,373400.0,745800.0,372400.0,4413.807201,348928.684222,20164.274726,173586.0,219000.0,6.0,30.0,24.0
4.0,4.0,Utah,118991.232273,56600.0,409700.0,353100.0,153000.0,428300.0,275300.0,1981.427075,145538.582002,5523.883025,18952.0,261970.0,2.0,7.0,5.0
10.0,10.0,NaN,332401.846333,346000.0,3890300.0,3544300.0,714900.0,1585900.0,871000.0,9805.471287,673602.940058,31861.375645,250573.0,652236.0,3.0,16.0,13.0


# Display HH/Emp Growth by Analysis Areas

In [24]:
df_AnalysisAreas = pd.read_csv(AnalysisAreasTable)

sdf_AnalysisAreas = pd.DataFrame.merge(sdf_AAid, df_AnalysisAreas, on='AreaID')

sdf_AANewHHEmp = pd.DataFrame.merge(sdf_AnalysisAreas, df_AASummaryTotals, on=['AreaID','ClassID'])
sdf_AANewHHEmp

,FID,Shape_Leng,Shape_Area,AreaID,ClassID,SHAPE,AnalysisAreaName,county_id,ClassDescription,ClassOrder,parcel_id,Acres,acres_redevelopable,acres_developable,job_spaces_orig,job_spaces_new,job_spaces_change,resunits_orig,resunits_new,resunits_change,job_spaces_undevelopable,job_spaces_remain,job_spaces_redeveloped_orig,job_spaces_redeveloped_new,job_spaces_developed,resunits_undevelopable,resunits_remain,resunits_redeveloped_orig,resunits_redeveloped_new,resunits_developed,density_hhemp_orig,density_hhemp_new,density_hhemp_change
0,0,599.400903,1.220382e+04,4,a4,"{'rings': [[[420271.3705000002, 4480517.338500...",Bluffdale,2.0,Single Family A4,4,2031.0,2707.052030,72.060894,807.318479,112,53,-59,1362,2470,1108,8.928213,44.949431,58.736514,0.0,0.0,3.995460,1346.229080,11.806696,91.0,1029.0,0.5,0.9,0.4
1,0,599.400903,1.220382e+04,4,a4,"{'rings': [[[420271.3705000002, 4480517.338500...",Bluffdale,4.0,Single Family A4,4,24.0,2.978770,0.000000,1.724645,0,0,0,0,2,2,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,2.0,0.0,0.7,0.7
2,1,1554.841774,8.561669e+04,4,a4,"{'rings': [[[420636.9475999996, 4481132.772299...",Bluffdale,2.0,Single Family A4,4,2031.0,2707.052030,72.060894,807.318479,112,53,-59,1362,2470,1108,8.928213,44.949431,58.736514,0.0,0.0,3.995460,1346.229080,11.806696,91.0,1029.0,0.5,0.9,0.4
3,1,1554.841774,8.561669e+04,4,a4,"{'rings': [[[420636.9475999996, 4481132.772299...",Bluffdale,4.0,Single Family A4,4,24.0,2.978770,0.000000,1.724645,0,0,0,0,2,2,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,2.0,0.0,0.7,0.7
4,2,254.967270,3.811046e+03,4,a4,"{'rings': [[[420671.3481999999, 4481261.932600...",Bluffdale,2.0,Single Family A4,4,2031.0,2707.052030,72.060894,807.318479,112,53,-59,1362,2470,1108,8.928213,44.949431,58.736514,0.0,0.0,3.995460,1346.229080,11.806696,91.0,1029.0,0.5,0.9,0.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12598,11368,17703.750849,2.944929e+06,10,h2,"{'rings': [[[414283.45440000016, 4552801.9068]...",Clearfield,1.0,Mixed Residential Suburban,35,7038.0,1808.511839,186.607310,159.943531,1645,15476,13831,8138,13870,5732,862.621959,717.171698,65.868814,7483.0,6414.0,124.255910,7260.839844,753.618220,3492.0,2993.0,5.4,16.2,10.8
12599,11369,10436.992442,3.292208e+06,10,h2,"{'rings': [[[412703.1934000002, 4553118.742900...",Clearfield,1.0,Mixed Residential Suburban,35,7038.0,1808.511839,186.607310,159.943531,1645,15476,13831,8138,13870,5732,862.621959,717.171698,65.868814,7483.0,6414.0,124.255910,7260.839844,753.618220,3492.0,2993.0,5.4,16.2,10.8
12600,11370,2229.618596,2.944859e+05,10,g4,"{'rings': [[[414942.4210000001, 4550112.225299...",Clearfield,1.0,Mixed-Use Metro,33,777.0,292.707193,62.602714,104.281059,2619,128816,126197,826,13541,12715,218.365347,1065.013156,1335.831246,47841.0,79692.0,9.981769,778.769282,37.821977,4784.0,7969.0,11.8,486.3,474.5
12601,11371,8681.643024,8.610622e+05,10,g4,"{'rings': [[[414009.90950000007, 4551759.66699...",Clearfield,1.0,Mixed-Use Metro,33,777.0,292.707193,62.602714,104.281059,2619,128816,126197,826,13541,12715,218.365347,1065.013156,1335.831246,47841.0,79692.0,9.981769,778.769282,37.821977,4784.0,7969.0,11.8,486.3,474.5


In [25]:
##create map centered on Salt Lake
#map_parcels_areas_add = gis.map('Salt Lake')
#
##expression to classify employment growth
#arcade_expression_jobspaceschange = ("var v = $feature.job_spaces_change;"
#                                     "if      (v<0    ) { return 'class0'; }"
#                                     "if      (v<2500 ) { return 'class1'; }"
#                                     "else if (v<5000 ) { return 'class2'; }"
#                                     "else if (v<10000) { return 'class3'; }"
#                                     "else              { return 'class4'; }")
#
##symbology for enrollment classes
#uv_jobspaceschange = [{"value":"class0", "label":"Job Decline"     , "symbol":{"type":"esriSFS","color":[  0,  0,  0,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
#                      {"value":"class1", "label":"0 to 2,500"      , "symbol":{"type":"esriSFS","color":[130,165,217,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
#                      {"value":"class2", "label":"2,500 to 5,000"  , "symbol":{"type":"esriSFS","color":[140,125,164,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
#                      {"value":"class3", "label":"5,000 to 10,000" , "symbol":{"type":"esriSFS","color":[149,85,111,168],  "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
#                      {"value":"class4", "label":"More than 10,000", "symbol":{"type":"esriSFS","color":[168,4,4,168],     "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}}]
#
#
##define sdf layer
#sdf_AANewHHEmp.spatial.plot(map_widget = map_parcels_areas_add,
#                            renderer_type='u-a', #'u-a' stands for uniqe value with arcade expression
#                            unique_values=uv_jobspaceschange,
#                            arcade_expression=arcade_expression_jobspaceschange,
#                            default_symbol="" #don't include an 'other' category
#                            )
#
##define map characteristics
#map_parcels_areas_add.layout.height='500px'
#map_parcels_areas_add.legend=True
#
##map title
#display(Markdown('<h2><center>Job Growth by Analysis Area</center></h2>'))
#
##display map
#map_parcels_areas_add

In [26]:
##create map centered on Salt Lake
#map_parcels_areas_addHH = gis.map('Salt Lake')
#
##expression to classify employment growth
#arcade_expression_resunitschange = ("var v = $feature.resunits_change;"
#                                    "if      (v<0   ) { return 'class0'; }"
#                                    "if      (v<1000) { return 'class1'; }"
#                                    "else if (v<2000) { return 'class2'; }"
#                                    "else if (v<5000) { return 'class3'; }"
#                                    "else             { return 'class4'; }")
#
##symbology for enrollment classes
#uv_resunitschange = [{"value":"class0", "label":"HH Decline"     , "symbol":{"type":"esriSFS","color":[  0,  0,  0,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
#                     {"value":"class1", "label":"0 to 1,000"     , "symbol":{"type":"esriSFS","color":[130,165,217,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
#                     {"value":"class2", "label":"1,000 to 2,000" , "symbol":{"type":"esriSFS","color":[140,125,164,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
#                     {"value":"class3", "label":"2,000 to 5,000" , "symbol":{"type":"esriSFS","color":[149, 85,111,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
#                     {"value":"class4", "label":"More than 5,000", "symbol":{"type":"esriSFS","color":[168,  4,  4,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}}]
#
#
##define sdf layer
#sdf_AANewHHEmp.spatial.plot(map_widget = map_parcels_areas_addHH,
#                            renderer_type='u-a', #'u-a' stands for uniqe value with arcade expression
#                            unique_values=uv_resunitschange,
#                            arcade_expression=arcade_expression_resunitschange,
#                            default_symbol="" #don't include an 'other' category
#                            )
#
##define map characteristics
#map_parcels_areas_addHH.layout.height='500px'
#map_parcels_areas_addHH.legend=True
#
##map title
#display(Markdown('<h2><center>Household Growth by Analysis Area</center></h2>'))
#
##display map
#map_parcels_areas_addHH

In [27]:
sdf_AA

,FID,objectid_1,city,county,city_lu_ty,gen_lu_typ,max_dua,plan_year,avg_lva,dua_group,rank,percentile,code,category,landuse,plan_sourc,data_sourc,Shape_Leng,Shape_Area,AnalysisAr,Analysis_1,SHAPE
0,0,18,Bluffdale,Salt Lake,,Residential SF,1.0,0.0,193071.312500,a - estate,0.729545,4 - 60-80,a4,,,,,599.400903,12203.821403,Bluffdale,a4,"{""rings"": [[[420271.3705000002, 4480517.338500..."
1,1,19,Bluffdale,Salt Lake,,Residential SF,1.0,0.0,282726.299479,a - estate,0.829545,5 - 80-100,a5,,,,,5673.401678,395913.547830,Bluffdale,a5,"{""rings"": [[[421136.0301000001, 4481104.8302],..."
2,2,20,Bluffdale,Salt Lake,,Residential SF,1.0,0.0,186557.414063,a - estate,0.722727,4 - 60-80,a4,,,,,1554.841774,85616.693206,Bluffdale,a4,"{""rings"": [[[420636.9475999996, 4481132.772299..."
3,3,21,Bluffdale,Salt Lake,,Residential SF,1.0,0.0,151498.921875,a - estate,0.636364,4 - 60-80,a4,,,,,254.967270,3811.045695,Bluffdale,a4,"{""rings"": [[[420671.3481999999, 4481261.932600..."
4,4,22,Bluffdale,Salt Lake,,Residential SF,1.0,0.0,318491.628125,a - estate,0.856818,5 - 80-100,a5,,,,,2588.398617,261893.287379,Bluffdale,a5,"{""rings"": [[[421931.24289999995, 4482241.10689..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11371,11371,0,Payson,Utah,Mixed Use Neighborhood,Mixed Use,12.0,2020.0,0.000000,,0.000000,,g1,g - mixed use,1 - single story,https://paysonutah.org/storage/2016/05/General...,https://paysonutah.org/storage/2016/05/General...,101.746609,647.000097,Payson,g1,"{""rings"": [[[437436.2742999997, 4432295.444599..."
11372,11372,0,Payson,Utah,Mixed Use Neighborhood,Mixed Use,12.0,2020.0,0.000000,,0.000000,,g1,g - mixed use,1 - single story,https://paysonutah.org/storage/2016/05/General...,https://paysonutah.org/storage/2016/05/General...,347.071602,6455.345939,Payson,g1,"{""rings"": [[[437517.15270000044, 4432410.1381]..."
11373,11373,0,Payson,Utah,Mixed Use Neighborhood,Mixed Use,12.0,2020.0,0.000000,,0.000000,,g1,g - mixed use,1 - single story,https://paysonutah.org/storage/2016/05/General...,https://paysonutah.org/storage/2016/05/General...,441.004580,8873.413053,Payson,g1,"{""rings"": [[[437390.2474999996, 4432411.0495],..."
11374,11374,0,Payson,Utah,Mixed Use Neighborhood,Mixed Use,12.0,2020.0,0.000000,,0.000000,,g1,g - mixed use,1 - single story,https://paysonutah.org/storage/2016/05/General...,https://paysonutah.org/storage/2016/05/General...,743.393043,29068.043955,Payson,g1,"{""rings"": [[[439725.0575000001, 4433219.461200..."
